<a href="https://colab.research.google.com/github/EASC/tvdi_hpc/blob/main/tp_final_fonnegra_solarte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/cursos/HPC/tp_final/code

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/cursos/HPC/tp_final/code'
/content/gdrive/MyDrive/cursos/HPC/tp_final/code


In [ ]:
from time import time, sleep
import multiprocessing as mp
import threading as thr


def calc_tvdi(q, ndvi, lst, min_ndvi, max_ndvi, delta, px, lock):
    #Listas para almacenar los valores de lst y ndvi para la regresión
    lst_regr = []
    ndvi_regr = []
    tmin = []

    #Eliminar valores nulos
    lock.acquire()
    nan_data = ~np.isnan(lst)
    nan_data2 = ~np.isnan(ndvi)
    nan_data = nan_data*nan_data2

    lst_C_reshape = lst[nan_data]
    ndvi_reshape = ndvi[nan_data]
    
    for v in np.arange(min_ndvi, max_ndvi, delta):
        dry_limit = extract_dry_limit(ndvi_reshape, lst_C_reshape, v, v+delta, px)
        
        if all(dry_limit):
            tmin.append(dry_limit[0])
            lst_regr.append(dry_limit[1])
            ndvi_regr.append(dry_limit[2])
    
    tmin = np.array(tmin)
    lst_regr = np.array(lst_regr)
    ndvi_regr = np.array(ndvi_regr)
    #Regresión lineal
    slope, intercept, r_val, p_val, std_err = stats.linregress(ndvi_regr, lst_regr)

    #Cálculo del TVDI
    tvdi = (lst - np.mean(tmin)) / (intercept + slope*ndvi - np.mean(tmin))
    lock.release()

    q.put(tvdi)
    
def main_tvdi(im_ndvi, im_lst, im_tvdi, lock, w_height=2000, min_ndvi=0, px=1, delta=0.01): # Cambio
    '''
    im_ndvi: nombre y ruta de imagen de entrada de NDVI
    im_lst: nombre y ruta de imagen de entrada de LST
    im_tvdi: nombre y ruta de salida de imagen TVDI
    w_height: alto de ventana para cálculo del índice
    min_ndvi: límite inferior de corte del histograma para calcular límite seco
    px: cantidad de pixeles de LST a tener en cuenta (límite seco)
    delta: define el paso durante la iteración
    '''
    q = mp.Queue()

    #cargar imágenes
    ndvi = load_img(im_ndvi)
    lst = load_img(im_lst)
    lst['array'][lst['array']<=0] = np.nan

    ################################
    array_tvdi = np.empty_like(ndvi['array'])
    
    #Parámetros
    max_ndvi = np.nanmax(ndvi['array'])
    process = list()
    for i in range(0, ndvi['rows']-w_height+1):
        if i%500==0:
            print(f'fila {i} a {i + w_height}: calculando deltas')
        ndvi_sub = ndvi['array'][i:i+w_height, :]
        lst_sub = lst['array'][i:i+w_height, :]
        t = thr.Thread(target=calc_tvdi, args=(q, ndvi_sub, lst_sub, min_ndvi, max_ndvi, delta, px, lock, ))
        t.start()
        process.append(t)

    for i in range(len(process)):

        # lock.acquire() # Cambio
        tvdi_sub = q.get()

        idx = w_height//2

        if i==0:
            array_tvdi[:idx+1, :] = tvdi_sub[:idx+1, :]
        elif i+w_height==ndvi['rows']:
            array_tvdi[i+idx:, :] = tvdi_sub[idx:, :]
        else:
            array_tvdi[i+idx, :] = tvdi_sub[idx, :]

        # lock.release() # Cambio
        
    for t in process: 
        t.join()

###########################################

    save_img(im_tvdi, array_tvdi, ndvi['rows'], ndvi['cols'], ndvi['geoTs'], ndvi['proj'])



fila 0 a 2000: calculando deltas
creando imagen
Ejecución finalizada en 20.084842443466187 segundos


In [ ]:
if __name__ == '__main__':
    a = time()
    P = mp.cpu_count()

    manager = mp.Manager()
    lock = manager.Lock()

    ndvi_fn = '../data/input/ndvi_2022209_500m.tif'
    lst_fn = '../data/input/lst_Celsius_2022209_500m.tif'
    output_tvdi = '../data/output/tvdi_2022209_500m_fn_op_2.tif'
    main_tvdi(ndvi_fn, lst_fn, output_tvdi, lock)
    b = time()
    print(f'Ejecución finalizada en {b - a} segundos')